In [524]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from scipy import stats
from scipy.stats import linregress

from datetime import datetime

Obtaining the datasets and adding the average row for the gas dataset

In [525]:
meralco_df = pd.read_csv('MonthlyPumpPrices.csv', parse_dates=['Date'])

fuelefficiency_df = pd.read_csv('CarsFuelEfficiency.csv')

ron95_data = meralco_df[meralco_df['Product'] == "Gasoline (RON95)"].sort_values('Date')
ron95_data = ron95_data[(ron95_data['Date'] >= '2020-01-01') & (ron95_data['Date'] <= '2025-01-01')]
ron95_data['Average Price'] = (ron95_data['Overall Range Min'] + ron95_data['Overall Range Max']) / 2

In [526]:
dry_season_color = 'rgba(255, 165, 0, 0.2)'   # Orange
wet_season_color = 'rgba(0, 191, 255, 0.2)'   # Light Blue

Monthly gas prices graph: (from 2020 to 2025)

In [527]:
price_line_color = '#DC143C'                  # Crimson
price_range_color = 'rgba(158, 27, 50, 0.5)'  # Dark Red with transparency

fig = go.Figure()

# Add seasonal background shapes first (so they appear behind the data)
for year in range(2019, 2025):
    # Dry season (Dec-May)
    fig.add_vrect(
        x0=max(datetime(year, 12, 1), datetime(2020, 1, 1)),
        x1=min(datetime(year+1, 6, 1), datetime(2024, 12, 31)),
        fillcolor=dry_season_color,
        layer="below",
        line_width=0
    )
    
    # Wet season (Jun-Nov)
    fig.add_vrect(
        x0=max(datetime(year, 6, 1), datetime(2020, 1, 1)),
        x1=min(datetime(year, 12, 1), datetime(2024, 12, 31)),
        fillcolor=wet_season_color,
        layer="below",
        line_width=0
    )

# Add price range (fill between)
fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Overall Range Max'],
    fill=None,
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Overall Range Min'],
    fill='tonexty',
    mode='lines',
    fillcolor=price_range_color,
    line=dict(width=0),
    name='Price Range',
    hovertemplate="<b>%{x|%b %Y}</b><br>Range: %{y:.2f}–%{text} pesos<extra></extra>",
    text=ron95_data['Overall Range Max'].round(2)
))

# Add average price line
fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Average Price'],
    mode='lines+markers',
    line=dict(color=price_line_color, width=3),
    marker=dict(size=8),
    name='Average Price',
    hovertemplate="<b>%{x|%b %Y}</b><br>Avg: %{y:.2f} pesos<extra></extra>"
))

# Calculate and add the regression line
x = (ron95_data['Date'].astype(np.int64) // 10**9)  # Convert datetime to timestamp (seconds since epoch)
y = ron95_data['Average Price']
slope, intercept, r_value, p_value, std_err = linregress(x, y)
regression_line = slope * x + intercept

fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=regression_line,
    mode='lines',
    line=dict(color='gray', dash='dot', width=2),
    name=f'Regression Line (R² = {r_value**2:.2f})',
    hovertemplate="<b>%{x|%b %Y}</b><br>Regression: %{y:.2f} pesos<extra></extra>"
))

# Layout customization
fig.update_layout(
    title={
        'text': "<b>Cost per Liter for Gasoline (RON95) Monthly Price Trends (2020-2024)</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'font': dict(size=18, color='#333333')
    },
    xaxis=dict(
        title="Date",
        range=['2020-01-01', '2024-12-31'],
        tickformat="%Y",
        dtick="M12",  # Yearly ticks
        showgrid=True,
        gridcolor='rgba(0,0,0,0.2)',
    ),
    yaxis=dict(
        title="Cost (pesos)",
        showgrid=True,
        gridcolor='rgba(0,0,0,0.2)',
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='#FAFAFA',
    paper_bgcolor='white',
    hovermode="x unified",
    margin=dict(t=100)
)

# Custom legend for seasons (requires adding dummy traces)
fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=dry_season_color),
    name='Dry Season (Dec-May)'
))

fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=wet_season_color),
    name='Wet Season (Jun-Nov)'
))

fig.show()

# Save as HTML
fig.write_html("graphs/gasoline_price_trends.html")


def plot_gasoline_boxplot() -> None:
    # Prepare data
    ron95_data['Year'] = ron95_data['Date'].dt.year
    fig = go.Figure()
    
    # Grouped boxplot by year
    for year, group in ron95_data.groupby('Year'):
        fig.add_trace(go.Box(
            y=group['Average Price'],
            name=str(year),
            boxpoints='all',  # Show individual points
            jitter=0.4,
            pointpos=0,
            fillcolor='rgba(220, 20, 60, 0.3)',  # Soft crimson fill
            marker_color=price_line_color,
            line=dict(width=1),
            marker=dict(size=5)
        ))

    # Layout styling
    fig.update_layout(
        title={
            'text': "<b>Distribution of Monthly Gasoline Prices per Liter (RON95) per Year (2020–2024)</b>",
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Year",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)'
        ),
        yaxis=dict(
            title="Cost (pesos per liter)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)'
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(t=100)
    )

    fig.show()

    # Optional: Save
    fig.write_html("graphs/gasoline_price_boxplot.html")

In [528]:
plot_gasoline_boxplot()

Monthly electricity prices graph (2020 to 2025)

In [529]:
## Relevant functions


def get_specific_kwh_df(kwh_consumption: int):
    meralco_df = pd.read_csv('MeralcoSummaryOfRates.csv')
    meralco_df['Date'] = pd.to_datetime(meralco_df['Year'].astype(str) + '-' + meralco_df['Month'].astype(str), format='%Y-%m')

    # Reorder the columns to make 'Date' the first column
    cols = ['Date'] + [col for col in meralco_df.columns if col != 'Date']
    meralco_df = meralco_df[cols]

    chosen_kwh_data = meralco_df[(meralco_df['kWh Consumption'] == kwh_consumption) & (meralco_df['For Lifeline Customers'] == False)]
    chosen_kwh_data = chosen_kwh_data[(chosen_kwh_data['Date'].dt.year >= 2020) & (chosen_kwh_data['Date'].dt.year <= 2024)]

    return chosen_kwh_data

def plot_specific_kwh_df(kwh_consumption: int = 200) -> None:
    df_to_plot = get_specific_kwh_df(kwh_consumption)

    line_color = 'rgba(0, 0, 255, 1)'            # Blue

    fig = go.Figure()

    # Add seasonal background shapes first (so they appear behind the data)
    for year in range(2019, 2025):
        # Dry season (Dec-May)
        fig.add_vrect(
            x0=max(datetime(year, 12, 1), datetime(2020, 1, 1)),
            x1=min(datetime(year+1, 6, 1), datetime(2024, 12, 31)),
            fillcolor=dry_season_color,
            layer="below",
            line_width=0,
        )
        
        # Wet season (Jun-Nov)
        fig.add_vrect(
            x0=max(datetime(year, 6, 1), datetime(2020, 1, 1)),
            x1=min(datetime(year, 12, 1), datetime(2024, 12, 31)),
            fillcolor=wet_season_color,
            layer="below",
            line_width=0,
        )

    # Add main line plot
    fig.add_trace(go.Scatter(
        x=df_to_plot['Date'],
        y=df_to_plot['Total Bill'],
        mode='lines+markers',
        line=dict(color=line_color, width=2),
        marker=dict(size=8),
        name='Electricity Cost',
        hovertemplate="<b>%{x|%b %Y}</b><br>Cost: %{y:.2f} pesos<extra></extra>"
    ))

    # Calculate and add the regression line
    x = (df_to_plot['Date'].astype(np.int64) // 10**9)  # Convert datetime to timestamp (seconds since epoch)
    y = df_to_plot['Total Bill']
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    regression_line = slope * x + intercept

    fig.add_trace(go.Scatter(
        x=df_to_plot['Date'],
        y=regression_line,
        mode='lines',
        line=dict(color='gray', dash='dot', width=2),
        name=f'Regression Line (R² = {r_value**2:.2f})',
        hovertemplate="<b>%{x|%b %Y}</b><br>Regression: %{y:.2f} pesos<extra></extra>"
    ))

    # Layout customization
    fig.update_layout(
        title={
            'text': f"<b>Meralco Cost per kW for {kwh_consumption} kWh Consumption (2020-2024, Non-Lifeline)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Date",
            range=['2020-01-01', '2024-12-31'],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)',
        ),
        yaxis=dict(
            title="Cost (pesos)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)',
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified"
    )
    fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=dry_season_color),
    name='Dry Season (Dec-May)'
    ))

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=wet_season_color),
        name='Wet Season (Jun-Nov)'
    ))

    fig.show()

    # Optional: Save as HTML
    fig.write_html(f"graphs/electricity_cost_{kwh_consumption}kWh.html")

def plot_kwh_boxplot(kwh_consumption: int = 200) -> None:
    df = get_specific_kwh_df(kwh_consumption)
    df['Year'] = df['Date'].dt.year

    fig = go.Figure()

    # Group by year and add a Box trace for each year
    for year, group in df.groupby('Year'):
        fig.add_trace(go.Box(
            y=group['Total Bill'],
            name=str(year),
            boxpoints='all',  # Show all points
            jitter=0.4,
            pointpos=0,
            marker=dict(size=5),
            line=dict(width=1),
            fillcolor='rgba(0,0,255,0.2)',
            marker_color='blue'
        ))

    fig.update_layout(
        title={
            'text': f"<b>Distribution of Meralco Cost per kWh for {kwh_consumption} kWh Consumption (2020-2024, Non-Lifeline)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis_title="Year",
        yaxis_title="Cost (pesos)",
        plot_bgcolor="#FAFAFA",
        paper_bgcolor="white",
        title_font=dict(size=18),
        xaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.2)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.2)')
    )

    fig.show()

    # Optional: Save
    fig.write_html(f"graphs/electricity_boxplot{kwh_consumption}kWh.html")


In [530]:
plot_specific_kwh_df(400)

In [531]:
plot_kwh_boxplot(400)

Comparing Fuel Efficiencies

In [532]:
def plot_fuel_efficiency_boxplot(data):
    # Filter the data for each category and handle BEV with Km/KWh
    ice_data = data[data['Category'] == 'ICE']['Combined (km/L)']
    hybrid_data = data[data['Category'] == 'Hybrid']['Combined (km/L)']
    bev_data = data[data['Category'] == 'BEV']['Combined (km/L)']

    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.5, 0.5]
    )
    
    # ICE & Hybrid boxplots
    fig.add_trace(go.Box(
        y=ice_data,
        name='ICE',
        boxmean='sd',
        marker=dict(color='red')
    ), row=1, col=1)
    
    fig.add_trace(go.Box(
        y=hybrid_data,
        name='Hybrid',
        boxmean='sd',
        marker=dict(color='blue')
    ), row=1, col=1)
    
    # BEV boxplot
    fig.add_trace(go.Box(
        y=bev_data,
        name='BEV',
        boxmean='sd',
        marker=dict(color='green')
    ), row=1, col=2)

    fig.update_layout(
        title={
            'text': (
                "<b>Efficiency Distribution by Car Type</b><br>"
            ),
            'y': 0.92,
            'x': 0.5,
            'xanchor': 'center',
            'font': dict(size=20, color='#333333')
        },
        yaxis_title="Fuel Efficiency (Km/L)",
        yaxis2_title="Energy Efficiency (Km/KWh)",
        showlegend=False,
        annotations=[
            dict(
                text='ICE & Hybrid Fuel Efficiency (Km/L)',
                x=0.07,
                y=1.1,
                showarrow=False,
                font=dict(size=16),
                xref="paper",
                yref="paper"
            ),
            dict(
                text='BEV Fuel Efficiency (Km/KWh)',
                x=0.9,
                y=1.1,
                showarrow=False,
                font=dict(size=16),
                xref="paper",
                yref="paper"
            ),
             dict(
                text="ICE: Internal Combustion Engine Vehicles",
                x=0,
                y=-0.17,
                showarrow=False,
                font=dict(size=12),
                xref="paper",
                yref="paper"
            ),
            dict(
                text="BEV: Battery Electric Vehicle",
                x=0,
                y=-0.23,
                showarrow=False,
                font=dict(size=12),
                xref="paper",
                yref="paper"
            )
        ]
    )

    # Show the plot
    fig.show()

    fig.write_html(f"graphs/fuelenergyefficiency_boxplot.html")

plot_fuel_efficiency_boxplot(fuelefficiency_df)

Comparing gas and electricity

In [533]:
def make_comparison_dataframe(km_l: float, km_kwh: float, kwh_consumption: int = 200):
    df_gas = ron95_data.copy()
    df_elec = get_specific_kwh_df(kwh_consumption)

    #print(df_elec)

    # Drop excess month in df_gas
    date_to_drop = pd.to_datetime('2025-01-01')
    df_gas = df_gas[df_gas['Date'] != date_to_drop]

    monthly_date_range = pd.date_range(start='2020-01-01', end='2024-12-31', freq='MS')
    # Create a new DataFrame with the monthly date range
    df_output = pd.DataFrame({'Date': monthly_date_range.date,
                              'Gas cost/km':None,
                              'Electricity cost/km':None,
                              'Difference': None})
    
    df_gas['Date'] = pd.to_datetime(df_gas['Date'])
    df_elec['Date'] = pd.to_datetime(df_elec['Date'])
    df_output['Date'] = pd.to_datetime(df_output['Date'])

    df_gas = df_gas.set_index('Date')
    df_elec = df_elec.set_index('Date')
    df_output = df_output.set_index('Date')

    # Calculate cost per km for each row
    for date in df_output.index:
        gas_price = df_gas.loc[date, 'Average Price']
        df_output.loc[date, 'Gas cost/km'] = gas_price / km_l  # PHP per km

        elec_price = df_elec.loc[date, 'Total Bill']
        df_output.loc[date, 'Electricity cost/km'] = elec_price / km_kwh  # PHP per km

    # Calculate the difference (Gas cost - Electricity cost)
    df_output['Difference'] = df_output['Gas cost/km'] - df_output['Electricity cost/km']
    df_output = df_output.reset_index()
    return df_output

# toyota vios (15) VS byd ATTO 3 (6.25)
"""
test data
30, 40, 50
6.25, 7, 7.5
"""

'\ntest data\n30, 40, 50\n6.25, 7, 7.5\n'

Plotting the dataframe comparison

In [534]:
def plot_comparison_dataframe(km_l: float, km_kwh: float, kwh_consumption: int = 200):
    df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)

    # Color definitions
    gas_color = '#EF553B'  # Red
    electricity_color = '#636EFA'  # Blue

    fig = go.Figure()

    ## vertical lines
    annotation_y = 0.8
    annotation_font = 10
    line_width = 2
    vline_color = 'black'

    major_events = {
        'Luzon ECQ Start': datetime(2020, 3, 1),
        'ECQ to MECQ': datetime(2021, 10, 1),
        'Russia-Ukraine War': datetime(2022, 2, 1),
        'ERC Order: Energy Costs': datetime(2024, 6, 1)
    }

    for event in major_events.keys():
        fig.add_shape(
            type='line',
            x0=major_events[event],
            x1=major_events[event],
            y0=annotation_y,
            y1=4.1,
            line=dict(color=vline_color, width=line_width, dash="dash"),
        )
        fig.add_annotation(
            x=major_events[event],
            y=annotation_y,
            xref="x",
            showarrow=False,
            text=event,
            textangle=-55,
            xanchor='left',
            yanchor='bottom',
            font=dict(color=vline_color, size=annotation_font),
        )

    # Add seasonal background shapes first (so they appear behind the data)
    for year in range(2019, 2025):
        # Dry season (Dec-May)
        fig.add_vrect(
            x0=max(datetime(year, 12, 1), df['Date'].min()),
            x1=min(datetime(year+1, 6, 1), df['Date'].max()),
            fillcolor=dry_season_color,
            layer="below",
            line_width=0,
            name='Dry Season (Dec-May)',
        )
        
        # Wet season (Jun-Nov)
        fig.add_vrect(
            x0=max(datetime(year, 6, 1), df['Date'].min()),
            x1=min(datetime(year, 12, 1), df['Date'].max()),
            fillcolor=wet_season_color,
            layer="below",
            line_width=0,
            name='Wet Season (Jun-Nov)',
        )

    # Add Gas line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Gas cost/km'],
        mode='lines+markers',
        line=dict(color=gas_color, width=2),
        marker=dict(size=8),
        name='Gas Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>Gas Cost: PHP %{y:.3f}<extra></extra>"
    ))

    # Add Electricity line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Electricity cost/km'],
        mode='lines+markers',
        line=dict(color=electricity_color, width=2),
        marker=dict(size=8),
        name='Electricity Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>Electricity Cost: PHP %{y:.3f}<extra></extra>"
    ))

    fig.update_layout(
        title={
            'text': "<b>Gas vs. Electricity Cost per Kilometer (2020-2025)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Date",
            range=[df['Date'].min(), df['Date'].max()],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)',
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified"
    )

    # Add seasonal legend entries (as in first plot)
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=dry_season_color),
        name='Dry Season (Dec-May)',
    ))

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=wet_season_color),
        name='Wet Season (Jun-Nov)',
    ))

    fig.add_annotation(
        x=0.9,
        y=-0.18,
        xref="paper",
        yref="paper",
        align="left",
        text=f"- Gas cost calculated with {km_l} km/L<br>- Electricity cost calculated with <br>  {km_kwh} km/kWh and {kwh_consumption} kWh consumption",
        showarrow=False,
        font=dict(size=11, color="#666666")
    )

    # Show the plot
    fig.show()
    fig.write_html(f"graphs/gas_vs_electricity.html")

def plot_comparison_boxplot(km_l: float, km_kwh: float, kwh_consumption: int):
    df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)
    df['Year'] = df['Date'].dt.year


    long_df = df[['Year', 'Gas cost/km', 'Electricity cost/km']].melt(
        id_vars='Year',
        var_name='Type',
        value_name='Cost/km'
    )
    long_df['Type'] = long_df['Type'].replace({
        'Gas cost/km': 'HEV',
        'Electricity cost/km': 'EV'
    })


    gas_color = '#EF553B'
    electricity_color = '#636EFA'
    color_map = {'HEV': gas_color, 'EV': electricity_color}

    fig = go.Figure()

    annotation_y = 0.8
    annotation_font = 10
    line_width = 2
    vline_color = 'black'

    for (year, vehicle_type), group in long_df.groupby(['Year', 'Type']):
        fig.add_trace(go.Box(
            y=group['Cost/km'],
            x=[str(year)] * len(group),
            name=vehicle_type,
            marker_color=color_map[vehicle_type],
            boxpoints='all',
            jitter=0.4,
            pointpos=0,
            line=dict(width=1),
            marker=dict(size=6),
            legendgroup=vehicle_type,
            showlegend=bool(year == long_df['Year'].min())
        ))

    fig.update_layout(
        title={
            'text': (
                "<b>Yearly Cost per km Distribution (HEV vs. EV)</b><br>"
                f"<sup>Gas: {km_l} km/L &nbsp;&nbsp;&nbsp;&nbsp;Electricity: {km_kwh} km/kWh at {kwh_consumption} kWh</sup>"
            ),
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'font': dict(size=20, color='#333333')
        },
        xaxis=dict(
            title="Year",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)'
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.2)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified",
        width=1800,
        height=1000
    )

    fig.show()
    fig.write_html(f"graphs/HEV_vs_EV_bvoxplot_{km_kwh}.html")

In [535]:
plot_comparison_boxplot(20.42, 6.23, 400)

In [536]:
plot_comparison_dataframe(20.42, 6.23, 400)

Statistical Test

In [537]:
def add_season_column(df):
    def assign_season(date):
        month = date.month
        if month == 12 or (1 <= month <= 5):  # November to May
            return "Dry"
        else:
            return "Wet"
    
    for date in df['Date']:
        df['Season'] = df['Date'].apply(assign_season)
        return df

def seasonality_test(_df: pd.DataFrame, column_to_check: str, time: str):
    res = []

    df = add_season_column(_df.copy())
    df = df[df['Date'].dt.year != 2025]
    
    if time.lower() == 'month':
        for i in df['Date'].dt.month.unique():
            time = df['Date'].dt.month
            res.append(df[time == i][column_to_check].values)
    elif time.lower() == 'year':
        for i in df['Date'].dt.year.unique():
            time = df['Date'].dt.year
            res.append(df[time == i][column_to_check].values)
    else:
        for i in df['Season'].unique():
            season = df['Season']
            res.append(df[season == i][column_to_check].values)

    
    return stats.kruskal(*res)

def save_all_kruskal(time: str):
    r1 = seasonality_test(ron95_data, 'Average Price', time)
    r2 = seasonality_test(get_specific_kwh_df(300), 'Total Bill', time)
    r3 = seasonality_test(make_comparison_dataframe(40, 7, 300), 'Difference', time)
    df = pd.DataFrame({
        'Seasonality': [time, time, time],
        'Dataset': ['Gas Avg Price', 'Meralco Rates', f'Gas VS Electricity Cost/km Difference'],
        'H Statistic': [r1.statistic, r2.statistic, r3.statistic],
        'P Value': [r1.pvalue, r2.pvalue, r3.pvalue],
    })
    return df

stat_results = pd.concat([save_all_kruskal('Season'), 
                          save_all_kruskal('Month'), 
                          save_all_kruskal('Year')], ignore_index=True)

output_html_path = "tables/kruskal.html"
with open(output_html_path, 'w', encoding='utf-8') as f:
    f.write(stat_results.to_html(index=False, escape=True))
    print(f"HTML table saved to {output_html_path}")

HTML table saved to tables/kruskal.html


Nutshell plot

In [538]:
def graph_REAL_nutshell_plot(km_l: float, km_kwh: float, kwh_consumption: int):
    ## 18, 7, 300, 40
    nutshell_df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)

    gas_color = '#dc267f'  # Red
    electricity_color = '#648fff'  # Blue
    vline_color = '#ffb000'
    text_color = '#FFFFFF'
    bg_color = '#1B181C'

    fig = go.Figure()
    fig.update_layout(width=1600,height=900)
    fig.update_layout(
        font=dict(
            color=text_color,
            family="Roboto",
            size=20,
        )
    )

    line_width = 3

    ## vertical lines
    luzon_lockdown = datetime(2020, 3, 1)
    lockdown_reduced = datetime(2021, 10, 1)
    ukraine_invasion = datetime(2022, 2, 1)
    erc_order = datetime(2024, 6, 1)
    y_val = max(max(nutshell_df['Gas cost/km']), max(nutshell_df['Electricity cost/km']))
    fig.update_layout(
        shapes=[
            dict(
                type="line",
                x0=luzon_lockdown,
                x1=luzon_lockdown,
                y1=y_val,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=lockdown_reduced,
                x1=lockdown_reduced,
                y1=y_val,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=ukraine_invasion,
                x1=ukraine_invasion,
                y1=y_val,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
        ],
    )

    # Add Gas line plot
    fig.add_trace(go.Scatter(
        x=nutshell_df['Date'],
        y=nutshell_df['Gas cost/km'],
        mode='lines+markers',
        line=dict(color=gas_color, width=3),
        marker=dict(size=12),
        name='',
    ))

    fig.add_trace(go.Scatter(
        x=nutshell_df['Date'],
        y=nutshell_df['Electricity cost/km'],
        mode='lines+markers',
        line=dict(color=electricity_color, width=3),
        marker=dict(size=12),
        name='',
    ))
    
    fig.update_layout(
        title={
            'text': "",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18)
        },
        xaxis=dict(
            title="Date",
            range=[nutshell_df['Date'].min(), nutshell_df['Date'].max()],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False,
            showline=True,
            linecolor='white',
            linewidth=2,
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=False,
            showline=True,
            linecolor='white',
        ),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
        xaxis_title="",
        yaxis_title="",
        hovermode="x unified",
    )

    for date in nutshell_df['Date']:
        month = date.month
        if month != 1:
            continue
        gas_cost = nutshell_df.loc[nutshell_df['Date'] == date]['Gas cost/km']
        elec_cost = nutshell_df.loc[nutshell_df['Date'] == date]['Electricity cost/km']
        
        #val = max(elec_cost.iloc[0], gas_cost.iloc[0])
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=date, y0=0.2,  # Start at the x-axis (y=0)
                x1=date, y1=y_val,  # End at the data point
                xref="x", yref="y",
                line=dict(color="rgba(211, 211, 211, 0.5)", width=2)
            )
        )

    # Show the plot
    fig.show()

graph_REAL_nutshell_plot(20.42, 6.23, 400)

In [539]:
#make_comparison_dataframe(40, 7, 200)

In [540]:
def graph_nutshell_plot(km_l: float, km_kwh: float, kwh_consumption: int):
    ## 18, 7, 300, 40
    nutshell_df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)

    gas_color = '#dc267f'  # Red
    electricity_color = '#648fff'  # Blue
    vline_color = '#ffb000'
    text_color = '#FFFFFF'
    bg_color = '#1B181C'

    fig = go.Figure()
    fig.update_layout(width=900)
    fig.update_layout(
        font=dict(
            color=text_color,
            family="Roboto",
        )
    )

    line_width = 1

    ## vertical lines
    luzon_lockdown = datetime(2020, 3, 1)
    lockdown_reduced = datetime(2021, 10, 1)
    ukraine_invasion = datetime(2022, 2, 1)
    erc_order = datetime(2024, 6, 1)
    annotation_y = 1.75
    annotation_font = 10
    fig.update_layout(
        shapes=[
            dict(
                type="line",
                x0=luzon_lockdown,
                x1=luzon_lockdown,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=lockdown_reduced,
                x1=lockdown_reduced,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=ukraine_invasion,
                x1=ukraine_invasion,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=erc_order,
                x1=erc_order,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
        ],
        annotations=[
            dict(
                x=luzon_lockdown,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="Luzon ECQ Start",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            ),
            dict(
                x=lockdown_reduced,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="ECQ to MECQ",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            ),
            dict(
                x=ukraine_invasion,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="Russia-Ukraine War",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            ),
            dict(
                x=erc_order,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="ERC Order: Energy Costs",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            )
        ],
    )

    # Add Gas line plot
    fig.add_trace(go.Scatter(
        x=nutshell_df['Date'],
        y=nutshell_df['Gas cost/km'],
        mode='lines+markers',
        line=dict(color=gas_color, width=2),
        marker=dict(size=8),
        name='HEV Gas Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>HEV Gas Cost: PHP %{y:.3f}<extra></extra>",
    ))

    fig.add_trace(go.Scatter(
        x=nutshell_df['Date'],
        y=nutshell_df['Electricity cost/km'],
        mode='lines+markers',
        line=dict(color=electricity_color, width=2),
        marker=dict(size=8),
        name='EV Electricity Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>EV Electricity Cost: PHP %{y:.3f}<extra></extra>",
    ))
    
    fig.update_layout(
        title={
            'text': "<b>Gas (HEV) VS Electricity (EV) Cost per Kilometer (2020-2025)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18)
        },
        xaxis=dict(
            title="Date",
            range=[nutshell_df['Date'].min(), nutshell_df['Date'].max()],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False,
            showline=True,
            linecolor='white'
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=False,
            showline=True,
            linecolor='white',
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
        hovermode="x unified",
    )

    for date in nutshell_df['Date']:
        month = date.month
        if month != 1:
            continue
        gas_cost = nutshell_df.loc[nutshell_df['Date'] == date]['Gas cost/km']
        elec_cost = nutshell_df.loc[nutshell_df['Date'] == date]['Electricity cost/km']
        
        val = max(elec_cost.iloc[0], gas_cost.iloc[0])
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=date, y0=0.2,  # Start at the x-axis (y=0)
                x1=date, y1=val,  # End at the data point
                xref="x", yref="y",
                line=dict(color="lightgray", width=1)
            )
        )

    fig.add_annotation(
        x=0.8,
        y=-0.27,
        xref="paper",
        yref="paper",
        align="left",
        text=f"""
        Gas cost calculated with {km_l} km/L<br>
        Electricity cost calculated with {km_kwh} km/kWh and {kwh_consumption} kWh household consumption<br>
        Sources: DoE NCR Prevailing Pump Prices, Meralco Rates
        """,
        showarrow=True,
        font=dict(size=11, color=text_color, family="Roboto Condensed")
    )

    # Show the plot
    fig.show()

graph_nutshell_plot(40, 7, 200)